Translating a Pedestrian Detector from MATLAB to PYTHON

James Alex Clark
Graduate Student
Center for Complex Systems and Brain Sciences
Florida Atlantic University
Email: jamesclark2016@fau.edu


Introduction

Self-driving automobiles are the wave of the future.  They already exist in prototype form,and are currently being implemented in cities around the world.  For the safety of the public,these cars need to have the ability to detect pedestrians.  Here I take a pre-written pedestriandetector algorithm, written in MATLAB, and translate it over to PYTHON.This  project  is  part  of  a  course  in  Machine  Perception  and  Cognitive  Robotics.   Thepurpose of this assignment is to familiarize myself with two programming languages, whilealso learning the steps and formulae involved in creating a pedestrian detector.I went through each line of MATLAB code individually, and used a combination of helpmenus (from both MATLAB and PYTHON) and internet searches to determine what theline of code is doing, and how best to achieve the same result in PYTHON. The result in apedestrian detector that should be able to operate in PYTHON.

Differences between MATLAB and PYTHON

The formulae necessary for creating a pedestrian detection algorithm vary from MATLABto PYTHON. In some cases the variation is very minor.  For example, the code to select all the columns for a particular row (r) of a matrix in MATLAB looks very similar to the equivalent code in PYTHON.
 
 MATLAB: A(r,:); 
 PYTHON: A[r,];
 
In  other  cases  the  difference  in  the  code  is  more  noticeable.   In  general,  the  code  to manipulate matrices is significantly more verbose in PYTHON than it is in MATLAB. 
 
Here is a list of common matrix commands in MATLAB.
Transpose = A’
Multiply = A*B
Concatenate = [A B]

And, here is a list of the same commands in PYTHON. 
Transpose = A.transpose()
Multiply = matrixmultiply(A,B)
Concatenateconcatenate((A,B),axis=1)

These examples make it clear that MATLAB is a more efficient language for matrix manipu-lation than PYTHON.

Lets break down the Pedestrian Detector Algorithm to see how it works.

The first thing we need is to import the necessary modules in PYTHON to do the various types of calculations and matrix manipulations we will need.  The code below will import all the necessary pieces, and in some cases gives them abbreviated names.

In [ ]:
from pylab import *
import numpy as np
import scipy as sci
import scipy.io as sio
from scipy.io import loadmat
import matplotlib
import random
from numpy import dot as matrixmultiply

The data to be analyzed by this algorithm contains a variety of images, some that contain a pedestrian and some that do not.  This data is imported from a file that has separate matrices for each category of data.  So, after importing we separate the data into its two relevant sections.  C1 contains the images with a pedestrian, and C2 contains the images without a pedestrian.

In [ ]:
data = loadmat('Daimler_ped_data.mat');
c1=data['c1']
c2=data['c2']

Before we proceed, we need to create three different functions that will be used in the program.  

The first is named im2double.  This function takes the values in the inputed image and scales them down to a range from 0 to 1.  It does this by turning the image matrix into a single row of pixle values, finding the min and max of those values, subtracting each value from the min, and dividing the result by the max-min for the entire image.

The second function is named af (short for activation function).  This function is a simple mathematical transformation that takes in an input called weighted_sum, and calculates an exponent by raising e to the negative value of the input, adding 1 to that result, and finding the inverse of that result.  We use the ./ function, because we will be applying this function elementwise to a matrix of values.  Note that when weighted_sum is a large positive number, then af will be close to 1, and when weighted_sum is a large negative number, af will be close to 0.

The third function is the most complex of the three.  It is called network_output, and it takes in the inputs: neural_input, hidden_weights, and output_weights.  

In [ ]:
def im2double(im):
    min_val = np.min(im.ravel())
    max_val = np.max(im.ravel())
    out = (im.astype('float') - min_val) / (max_val - min_val)
    return out

def af(weighted_sum):

    af = 1/(1+math.exp(-weighted_sum));  		# Logistic / Sigmoid Function

def network_output (neural_input,hidden_weights,output_weights):
    niconc = np.concatenate(neural_input,1);
    niconchw = matrixmultiply(niconc,hidden_weights);
    network_output = af(matrixmultiply(concatenate(af(niconchw),1),output_weights));

Once the data is imported, we reshape the data so that each image is its own column.  There are 5000 images in C1 and 4800 images in C2.  Each image contains 648 pixels.

CREATE THE MATRIX OF IMAGES 'pattern':
Now that the images have the same number of rows, we can concatenate them into one large data matrix.  This will create a 648 x 9800 matrix where each column is an image, and each image has 648 pixels. Then we transpose that matrix so that each image is now a row of length 648, and there are 9800 rows (one for each image).

CREATE THE MATRIX OF CORRECT RESPONSES 'category':
Finally, we must create our martix that contains the correct responses (or ground truth) for each image.  To do this, we first create a categoryc1 matrix of all ones.  It will have one row, and 5000 columns - one column for each entry in the C1 dataset (i.e. a one for each image with a pedestrian in it).  Second, we create a categoryc2 matrix of all zeros.  This matrix will have one row, and 4800 columns - one column for each entry in the C2 dataset (i.e. a zero for each element in the C2 dataset).  NOTE: In PYTHON, shape[1] = number of columns, shape[0] = number of rows.  We then take these two matrices and combine them into one long single-row matrix, and transpose that matrix into a long column.  This column matrix will now have 5000 ones and 4800 zeros, and each entry will correspond to the correct response that the algorithm should give when it tries to detect a pedestrian in an image from the original imported data.

In [ ]:
c1=c1.reshape(648,5000)
c2=c2.reshape(648,4800)
pattern=np.concatenate((im2double(c1), im2double(c2)),axis=1)
pattern=pattern.transpose();

categoryc1=ones((1,c1.shape[1]));
categoryc2=0.*ones((1,c2.shape[1]));
category=np.concatenate((categoryc1, categoryc2), axis=1);
category=category.transpose();

When we run the detector, we will want to randomly select from the images we have available.  This is achieved by creating a matrix r that is a random list of numbers from 1 to 9800.  Then we use this random list of numbers to reorder both the pattern and category matrices.  This will give us the same random shuffling of images in the 'pattern' and the 'category' matrix, so that the ground truth for each image is still preserved.

In [ ]:
r=np.random.permutation(size(pattern,0));

pattern=pattern[r,];
category=category[r,];

The next step is to separate our - now randomized - data into three parts: training data, testing data, and cross validation data.  We do this by simply taking the first third for training, the second third for testing, and the final third for cross validation.  Both 'pattern' and 'category' are divided in the same way, so the ground truth for each image in each part is preserved.  At this point, we have the following shapes for our various matrices:
Trainpattern: (3266, 648)
Traincategory: (3266, 1)
Testpattern: (3265, 648)
Testcategory: (3265, 1)
Validpattern: (3267, 648)
Validcategory: (3267, 1)
The number of rows is not a perfect 1/3 division.  This is due to the calculation we put in place to divide the randomized dataset.  Importantly, the discrepency (+/- one example) should not pose any problems for training, testing, or validation datasets.

In [ ]:
trainpattern=pattern[:pattern.shape[0]/3,];
testpattern=pattern[1+(pattern.shape[0]/3):2*(pattern.shape[0]/3),];
validpattern=pattern[1+(2*(pattern.shape[0]/3)):pattern.shape[0],];
traincategory=category[:pattern.shape[0]/3,];
testcategory=category[1+(pattern.shape[0]/3):2*(pattern.shape[0]/3),];
validcategory=category[1+(2*(pattern.shape[0]/3)):pattern.shape[0],];

For each set of images (train, test, and cross validation) we need to add the bias values (all ones) to the dataset.  Now our pattern matrices have the following shapes:
Trainpattern: (3266, 649)
Testpattern: (3265, 649)
Validpattern: (3267, 649)


In [ ]:
biastrain=ones((size(trainpattern,0),1));
trainpattern = np.concatenate((trainpattern, biastrain),axis=1);
biastest=ones((size(testpattern,0),1));
testpattern = np.concatenate((testpattern, biastest),axis=1);
biasvalid=ones((size(validpattern,0),1));
validpattern = np.concatenate((validpattern, biasvalid),axis=1);

Next, we define a few numeric variables that will be used in the calculations involved in training the pedestrian detector network:
 - n1 is the number of columns in the trainpattern matrix.  Recall that there are 648 pixels for each image, and that   we added a bias number to each image.  So, the number of columns in trainpattern is now 649.  
 - n2 is defined as 2.  
 - n3 takes the value of the number of columns in the traincategory matix.  In this case, there is just one column and   it will contain a zero if the image does not contain a pedestrian, and a 1 if the image does contain a pedestrian.
 - w1 is a matrix that will contain the weights for the first layer of the Neural Network.  It is initialized as a random matrix of elements in the range 0 to 1, with dimensions: #rows = n1 = 649, #columns = n2-1 = 1, that is then multiplied by 2 elementwise, and the resulting matrix is subtracted elementwise from 1.  This results in a random matrix with elements between -1 and 1.
 - w2 is a matix that will contain the weights for the second layer of the Neural Network.  It is initialized in a similar format to w1, but it is shaped with the following dimensions: #rows = n2 = 2, #columns = n3 = 1.
  - dw1 is initialized as a matrix of zeros of the same shape as w1
  - dw2 is initialized as a matrix of zeros of the same shape as w2
  - L is the learning rate for the Neural Network
  - M is the rate of momentum for the Neural Network
  - errorplot is a matrix that will be updated to create a graph of the error as the network trains.  This is initialized as a blank matrix.

In [ ]:
n1 = size(trainpattern,axis=1);
n2 = 2;
n3 = size(traincategory,axis=1);

w1 = 0.001*(1-2*np.random.rand((n1,n2-1)));
w2 = 0.001*(1-2*np.random.rand((n2,n3)));

dw1 = zeros(w1.shape);
dw2 = zeros(w2.shape);

L = 0.0001;
M = 0.8;

errorplot=[];

Now that all our valriables have been initialized, and our data is in the appropriate format, we can write the loop that will actually train the Neural Network.  The first part of the loop does the mathematical calculations necessary to update the Network paramters.  It will run 5000 times.

act1 is the variable that will store the activation values for the first layer of the NN.  It is computed by taking a few simple steps. 
1. Create a scaled down version of the the training examples (scaled by b=0.1).  This will create a matrix with dimensions the same as trainpattern (3266 x 649)
2. Multiply that new matrix with w1.  Looking at the shapes of the matrices we can continue to track the size of the resulting matrix: 3266 x 649 * 649 x 1 = 3266 x 1.  So, act1 now has dimensions 3266 x 1. This means that act1 will have a single value for each of the images being used for training.
3. Apply the activation function to the act1 matrix. This produces the activation values for the first layer of the NN. 3266 x 1 matrix of activation values.
4.  Add a bias factor to the resulting matrix.  This is just a column of ones.  Now act1 will have dimensions: 3266x2

The next part of the loop takes these activation values from the first layer and uses them to compute the activation values from the second layer of the NN.  There is only one step in this process.
5. Multiply the act1 matrix with the weights for the second layer of the NN.  This produces the activation values for the second layer of the network.  3266x2 * 2x1 = 3266x1
This matrix will contain the output values (ranging from 1 to 0) for each of the 3266 images in the trainpattern matrix.

Once the NN has computed its prediction for each image in the traiing set, the error is caluclated.
6.  Subtract the act2 matrix from the traincategory matrix.  Remember that traincategory is also a 3266x1 matrix, but it is made of 1s and 0s with a 1 corresponding to the images where a pedestrian is present, and a 0 corresponding to the images where there is no pedestrian.  So, the error matrix is also 3266x1 in size.
7.  Each element in this error is then squared, and the sum for the entire vector is calculated.  This is the total squared error for all the image predictions that the NN makes.

Next we calculate the delta for each of the weight matrices.  These will helpus make the adjustments to the weight matrices so that they can be updated in the correct way to make the NN more accurate as it trains.
8.  The delta_w2 is just an elementwise multiplication of the error matrix, the act2 matrix, and a matrix made of all the act2 elements after they have been subtracted from 1.  This produces a matrix delta_w2 of size 3266x1.
9.  The delta_w1 is a bit more complicated.  First we do an elementwise multiplication with act1, the matrix of values 1-act1, and the result of mulitplying delta_w2 with the transpose of w2.  Delta_w2 is 3622x1, and w2 is 2x1, so w2 transpose is 1x2.  So, delta_w2 * w2 transpose will produce a 3266x2 matrix. This has the same dimensions as act1 and 1-act, so an elementwise multiplication is possible.  So delta_w1 is 3266x2 matrix.
10.  Next we remove the final column from delta_w1.  This was the column that corresponded to the bias elements added to the w1.
Next we create and update the dw1 and dw2 matrices.
11.  dw1 is updatet as follows:
Mulitply the momentum value by dw1, add the results of the learning rate L multiplied by the product of the transpose of trainpattern (our original test data input, and has size 3266x649) and delta_w1 (size 3266x1).  This results in a matrix of size 649x1, and contains the adjustments to be made to the weights in the first layer of the NN, w1.
12.  dw2 is updated as follows:
Mulitply the momentum value by dw2, add the results of the learning rate L multiplied by the produce of the transpose of act1 (size 3266x2) and delta_w2 (size 3266x1).  So, dw2 will be a matrix of size 2x1 and it will contain the adjustments for the second layer of weights in the NN, w2.
13.  Finally, we add dw1 and dw2 to w1 and w2 respectively, and this is the adjustment made to the weights in our NN for one run of the loop.  This process will repeat 5000 times, adjusting the weights in both layers of the NN as it goes.

In [ ]:
for loop in range(1,5000):

    b=.1;
    
    act1 = matrixmultiply(b,trainpattern);
    act1 = matrixmultiply(act1,w1);
    act1 = np.concatenate((act1, biastrain),axis=1);
    act1 = af(act1)
    
    act2 = af(matrixmultiply((act1),w2));
    
    error = traincategory - act2;
    error = error**2;
    sse = error.sum(axis=0); 
    
    delta_w2 = error * act2 * (1-act2);
    delta_w1 = matrixmultiply(delta_w2,(w2.transpose()))*act1*(1-act1);

    delta_w1=scipy.delete(delta_w1,size(delta_w1,axis=1)-1,1);

    dw1 = L * trainpattern.transpose() * delta_w1 + M * dw1;
    dw2 = L * act1.transpose() * delta_w2 + M * dw2;
    
    w1 = w1 + dw1;
    w2 = w2 + dw2;

Now that we have seen the steps the algorithm takes to create and train the NN, lets look at the formula it is actually implementing.